In [ ]:
#hide
from nbdev.showdoc import show_doc

# 10-fold Cross Validation
> Code for training is the same as Notebook 05.

In [ ]:
from car_speech.fname_processing import load_fnames
from car_speech.pipeline import *

import string
from random import shuffle
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from sklearn.model_selection import train_test_split, KFold

## Configuration

In [ ]:
DATASET_TYPE = 'mixed' # 'digits' or 'letters' or 'mixed'

EPOCHS = 30

# load classified filenames
filenames = load_fnames('noise_levels/mixed_noise_levels/IDL.data')
num_samples = len(filenames)
print('number of files:', len(filenames))

# shuffle
shuffle(filenames)

fold_no = 1
acc_per_fold = []

number of files: 4274


In [ ]:
if DATASET_TYPE == 'digits':
    label_strings = np.array([str(num) for num in range(0,10)])
elif DATASET_TYPE == 'letters':
    label_strings = np.array(list(string.ascii_uppercase))
elif DATASET_TYPE == 'mixed':
    label_strings = np.array([str(num) for num in range(0,10)] + list(string.ascii_uppercase))

## Loop for 10-fold

In [ ]:
beg_ratio = 0
while beg_ratio < 0.9:
    test_beg = int(beg_ratio*num_samples)
    test_end = int((beg_ratio+0.1)*num_samples)
    
    fnames_copy = deepcopy(filenames)
    test_files = fnames_copy[test_beg:test_end]
    del fnames_copy[test_beg:test_end]
    
    val_len = len(test_files)
    val_files = fnames_copy[-val_len:]
    del fnames_copy[-val_len:]
    
    train_files = fnames_copy
    
    beg_ratio += 0.1
    
    train_files = shuffle_data(train_files)
    val_files = shuffle_data(val_files)
    test_files = shuffle_data(test_files)
    

    # Process data using the combined pipeline
    spectrogram_ds = preprocess_dataset(train_files, DATASET_TYPE)
    train_ds = spectrogram_ds
    val_ds = preprocess_dataset(val_files, DATASET_TYPE)
    test_ds = preprocess_dataset(test_files, DATASET_TYPE)
    print("Pipeline Completed")
    
    # batch
    batch_size = 64
    train_ds = train_ds.batch(batch_size)
    val_ds = val_ds.batch(batch_size)
    
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    train_ds = train_ds.cache().prefetch(AUTOTUNE)
    val_ds = val_ds.cache().prefetch(AUTOTUNE)
    
    # model
    for spectrogram, _ in spectrogram_ds.take(1):
        input_shape = spectrogram.shape
#     print('Input shape:', input_shape)
    num_labels = len(label_strings)

    norm_layer = preprocessing.Normalization()
    norm_layer.adapt(spectrogram_ds.map(lambda x, _: x))

    model = models.Sequential([
        layers.Input(shape=input_shape),
        preprocessing.Resizing(32, 32), 
        norm_layer,
        layers.Conv2D(32, 3, activation='relu'),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_labels),
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )
    
    # Train
    history = model.fit(
        train_ds, 
        validation_data=val_ds,  
        epochs=EPOCHS,
        callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
    )
    
    # Test
    test_audio = []
    test_labels = []

    for audio, label in test_ds:
        test_audio.append(audio.numpy())
        test_labels.append(label.numpy())

    test_audio = np.array(test_audio)
    test_labels = np.array(test_labels)
    
    y_pred = np.argmax(model.predict(test_audio), axis=1)
    y_true = test_labels

    test_acc = sum(y_pred == y_true) / len(y_true)
    print(f'Fold {fold_no} - Test set accuracy: {test_acc:.0%}')
    acc_per_fold.append(round(test_acc, 2))
    fold_no += 1

print(acc_per_fold)
print('mean score:', round(np.mean(acc_per_fold), 2))

Pipeline Completed
Epoch 1/30
54/54 [==============================] - 20s 365ms/step - loss: 3.4347 - accuracy: 0.0898 - val_loss: 3.1279 - val_accuracy: 0.1710
Epoch 2/30
54/54 [==============================] - 4s 69ms/step - loss: 3.0309 - accuracy: 0.1789 - val_loss: 2.6762 - val_accuracy: 0.2810
Epoch 3/30
54/54 [==============================] - 4s 70ms/step - loss: 2.6449 - accuracy: 0.2468 - val_loss: 2.3646 - val_accuracy: 0.3208
Epoch 4/30
54/54 [==============================] - 4s 71ms/step - loss: 2.3671 - accuracy: 0.2982 - val_loss: 2.1578 - val_accuracy: 0.4052
Epoch 5/30
54/54 [==============================] - 4s 73ms/step - loss: 2.1275 - accuracy: 0.3632 - val_loss: 2.0480 - val_accuracy: 0.4122
Epoch 6/30
54/54 [==============================] - 4s 73ms/step - loss: 1.9519 - accuracy: 0.3956 - val_loss: 1.9444 - val_accuracy: 0.4309
Epoch 7/30
54/54 [==============================] - 4s 69ms/step - loss: 1.8109 - accuracy: 0.4421 - val_loss: 1.8731 - val_accuracy:

54/54 [==============================] - 12s 214ms/step - loss: 3.4148 - accuracy: 0.0925 - val_loss: 3.0999 - val_accuracy: 0.2173
Epoch 2/30
54/54 [==============================] - 4s 69ms/step - loss: 2.9425 - accuracy: 0.1928 - val_loss: 2.6534 - val_accuracy: 0.2757
Epoch 3/30
54/54 [==============================] - 4s 70ms/step - loss: 2.5885 - accuracy: 0.2528 - val_loss: 2.3027 - val_accuracy: 0.3598
Epoch 4/30
54/54 [==============================] - 4s 70ms/step - loss: 2.3238 - accuracy: 0.3148 - val_loss: 2.1172 - val_accuracy: 0.3855
Epoch 5/30
54/54 [==============================] - 4s 70ms/step - loss: 2.0910 - accuracy: 0.3704 - val_loss: 2.0085 - val_accuracy: 0.4416
Epoch 6/30
54/54 [==============================] - 4s 70ms/step - loss: 1.9005 - accuracy: 0.4204 - val_loss: 1.8514 - val_accuracy: 0.4556
Epoch 7/30
54/54 [==============================] - 4s 70ms/step - loss: 1.7601 - accuracy: 0.4547 - val_loss: 1.7646 - val_accuracy: 0.4743
Epoch 8/30
54/54 [====

Epoch 3/30
54/54 [==============================] - 4s 70ms/step - loss: 2.6068 - accuracy: 0.2515 - val_loss: 2.3786 - val_accuracy: 0.3536
Epoch 4/30
54/54 [==============================] - 4s 70ms/step - loss: 2.3015 - accuracy: 0.3278 - val_loss: 2.1769 - val_accuracy: 0.3981
Epoch 5/30
54/54 [==============================] - 4s 70ms/step - loss: 2.0483 - accuracy: 0.3924 - val_loss: 2.0824 - val_accuracy: 0.4309
Epoch 6/30
54/54 [==============================] - 4s 70ms/step - loss: 1.8652 - accuracy: 0.4374 - val_loss: 2.0054 - val_accuracy: 0.4169
Epoch 7/30
54/54 [==============================] - 4s 70ms/step - loss: 1.7010 - accuracy: 0.4839 - val_loss: 1.9747 - val_accuracy: 0.4333
Epoch 8/30
54/54 [==============================] - 4s 70ms/step - loss: 1.5933 - accuracy: 0.5050 - val_loss: 1.8866 - val_accuracy: 0.4637
Epoch 9/30
54/54 [==============================] - 4s 70ms/step - loss: 1.4413 - accuracy: 0.5553 - val_loss: 1.9703 - val_accuracy: 0.4520
Epoch 10/30
5

In [ ]:
print("Done")
print(acc_per_fold)
print('mean score:', round(np.mean(acc_per_fold), 2))

Done
[0.5, 0.54, 0.5, 0.48, 0.55, 0.48, 0.52, 0.55, 0.45, 0.47]
mean score: 0.5
